In [21]:
import pandas as pd
from pycaret.time_series import *
import datetime

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

import warnings
warnings.filterwarnings("ignore")

In [22]:
# Cargar datos desde un archivo CSV
df = pd.read_csv("../data/scrap/demanda_tiempo_real.csv")

# Eliminar columnas innecesarias del DataFrame
df.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis=1, inplace=True)

# Convertir la columna "datetime" a tipo de dato datetime con zona horaria UTC y formato específico
df["datetime"] = pd.to_datetime(df["datetime"], utc=True, format='%Y-%m-%d %H:%M:%S%z')

# Crear una nueva columna "drop" que contiene "datetime" si los minutos no son múltiplo de 10, de lo contrario contiene "drop"
df["drop"] = df["datetime"].where(df["datetime"].dt.minute % 10 == 0, other="drop")

# Filtrar las filas donde "drop" no sea igual a "drop", eliminando las filas que no son múltiplo de 10 minutos
df = df[df["drop"] != "drop"]

# Eliminar la columna "drop" ya que ya no es necesaria
df.drop(["drop"], inplace=True, axis=1)

# Seleccionar y renombrar las columnas relevantes
df = df[["datetime", "value_Demanda real"]].rename(columns={"value_Demanda real": "demanda_real"})


In [3]:
# Crear una nueva columna "datetime" con una secuencia de fechas y horas
df["datetime"] = pd.date_range(datetime(2013, 12, 31, hour = 23, minute=0), periods = 486228, freq = '10min')

In [4]:
df.head()

,datetime,demanda_real
0,2013-12-31 23:00:00,24546
1,2013-12-31 23:10:00,24309
2,2013-12-31 23:20:00,24348
3,2013-12-31 23:30:00,24321
4,2013-12-31 23:40:00,24194


In [5]:
# Convertir la columna "datetime" a tipo de dato datetime y asignarla como índice del DataFrame
df.index = pd.to_datetime(df["datetime"])

In [6]:
# Inferir la frecuencia de los índices del DataFrame
df.index.freq = pd.infer_freq(df.index)
df.index

DatetimeIndex(['2013-12-31 23:00:00', '2013-12-31 23:10:00',
               '2013-12-31 23:20:00', '2013-12-31 23:30:00',
               '2013-12-31 23:40:00', '2013-12-31 23:50:00',
               '2014-01-01 00:00:00', '2014-01-01 00:10:00',
               '2014-01-01 00:20:00', '2014-01-01 00:30:00',
               ...
               '2023-03-31 11:20:00', '2023-03-31 11:30:00',
               '2023-03-31 11:40:00', '2023-03-31 11:50:00',
               '2023-03-31 12:00:00', '2023-03-31 12:10:00',
               '2023-03-31 12:20:00', '2023-03-31 12:30:00',
               '2023-03-31 12:40:00', '2023-03-31 12:50:00'],
              dtype='datetime64[ns]', name='datetime', length=486228, freq='10T')

In [7]:
# Eliminar la columna 'datetime' del DataFrame 'df' de forma inplace
df.drop(columns=['datetime'], inplace=True)
df.head()

,demanda_real
datetime,
2013-12-31 23:00:00,24546
2013-12-31 23:10:00,24309
2013-12-31 23:20:00,24348
2013-12-31 23:30:00,24321
2013-12-31 23:40:00,24194


In [8]:
# Visualizamos cuantos nulos hay
df.isnull().sum()

demanda_real    0
dtype: int64

In [9]:
# Renombramos la columna a "valores"
df.columns = ["valores"]

In [10]:
# Preparamos los settings de Pycaret
s = setup(df, fh = 3, fold = 5, session_id = 42)

In [11]:
# Buscamos y comparamos los mejores modelos
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:00:00
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [12]:
# Visualizamos nuestro mejor modelo
best

ARIMA(seasonal_order=(0, 1, 0, 2))

In [13]:
# Entrenamos un modelo con las caraterísticas del mejor modelo
final_best = finalize_model(best)


In [14]:
# Guardamos nuestro modelo que tanto nos ha costado entrenar
save_model(final_best, '../data/modelo/my_final_best_model')

Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 ForecastingPipeline(steps=[('forecaster',
                                                                                             TransformedTargetForecaster(steps=[('model',
                                                                                                                                 ARIMA(seasonal_order=(0,
                                                                                                                                                       1,
                                                                                                                                                       0,
                                                                                                                                                       2)))]))]))]))]),
 'my_fin

In [19]:
# Realizamos una predicción de 24 valores a partir de los datos con los que ha sido entrenados
predictions = predict_model(best, fh = 24)

In [20]:
# Guardamos la predicción para futura visualización
predictions.to_csv("../data/visualizacion/predicción.csv")

In [16]:
# Podemos cargar el modelo que hemos entrenado en esta ruta
loaded_model = load_model('../data/modelo/my_final_best_model')